### Created by yunsuxiaozi

#### This notebook is study from <a href="https://www.kaggle.com/code/ksevta/ps4e2-xgb-lgbm-0-92#Feature-Engineering-&-Processing">PS4E2: XGB / LGBM | 0.92+</a>. If you think my notebook is useful,please upvote for me.

## Import necessary libraries

In [ ]:
#necessary
import pandas as pd#导入csv文件的库
import numpy as np#进行矩阵运算的库
from sklearn.feature_extraction.text import TfidfVectorizer#导入tf-idf模型
from sklearn.decomposition import TruncatedSVD#基于奇异值分解的降维方法:截断SVD
#设置随机种子,保证模型可以复现
import random
import warnings#避免一些可以忽略的报错
warnings.filterwarnings('ignore')#filterwarnings()方法是用于设置警告过滤器的方法，它可以控制警告信息的输出方式和级别。

## Config

In [ ]:
#config
class Config():
    origin_path="/kaggle/input/obesity-or-cvd-risk-classifyregressorcluster/ObesityDataSet.csv"
    train_path='/kaggle/input/playground-series-s4e2/train.csv'
    test_path='/kaggle/input/playground-series-s4e2/test.csv'
    submission_path='/kaggle/input/playground-series-s4e2/sample_submission.csv'
    seed=2024
    num_folds=10
    TARGET_NAME ='NObeyesdad'

## preprocessor

In [ ]:
#一个处理数据的通用类,用于打playground比赛
class Preprocessor:#数据预处理的一个类
    def __init__(self,train_path=None,origin_path=None,test_path=None,TARGET_NAME ='target',seed=2024):
        self.seed=seed
        #设置随机种子,保证模型可以复现
        np.random.seed(self.seed)
        random.seed(self.seed)
        #需要预测的目标列
        self.TARGET_NAME=TARGET_NAME 
        
        if train_path!=None:#一般都有训练数据的路径
            self.train_df=pd.read_csv(train_path)
            self.train_df.drop(['id'],axis=1,inplace=True)
        else:#没有训练数据的路径就报错
            assert 0#assert 0==1?
        
        if origin_path!=None:#如果打算使用原始数据,没有原始数据也无所谓
            origin_df=pd.read_csv(origin_path)
            print(f"len(origin_df):{len(origin_df)}")
            #如果有原始数据,则把训练数据和原始数据拼接
            self.train_df=pd.concat((self.train_df,origin_df),axis=0)
            print(f"len(self.train_df):{len(self.train_df)}")
        self.train_df.fillna(method="ffill",inplace=True)
        self.train_df=self.train_df.drop_duplicates()
        print(f"len(self.train_df):{len(self.train_df)}")
            
        if test_path!=None:#一般都有训练数据的路径
            self.test_df=pd.read_csv(test_path)
            self.test_df.drop(['id'],axis=1,inplace=True)
            self.test_df.fillna(method="ffill",inplace=True)#用上一个数据来填充
        else:#没有训练数据的路径就报错
            assert 0#assert 0==1?
        #浮点数类型的连续型变量
        self.num_cols=[]
        #类别型变量,一般类别较少
        self.cat_cols=[]
        #一般就是像姓名那样不是类别型的字符串了
        self.str_cols=[]
        for col in self.train_df.columns:
            if (self.train_df[col].dtype!='float') and (col!=self.TARGET_NAME) and (self.train_df[col].nunique()<50):
                self.cat_cols.append(col)
            elif (self.train_df[col].dtype=='float') and (col!=self.TARGET_NAME):
                self.num_cols.append(col)
            elif (col!=self.TARGET_NAME):#一般就是普通的字符串了
                self.str_cols.append(col)
        print(f"num_cols:{self.num_cols}.\ncat_cols:{self.cat_cols}.\nstr_cols:{self.str_cols} ")
        
    #计算两组变量的皮尔逊相关系数
    def pearson_corr(self,x1,x2):
        #x1,x2 np.array
        eps=1e-15
        mean_x1=np.mean(x1)
        mean_x2=np.mean(x2)
        std_x1=np.std(x1)
        std_x2=np.std(x2)
        pearson=np.mean((x1-mean_x1)*(x2-mean_x2))/(std_x1*std_x2+eps)
        return pearson
    
    #用于进行探索性数据分析的函数.
    def EDA(self,):
        print("num_cols VS cat_cols:")
        for num_col in self.num_cols:
            for cat_col in self.cat_cols:
                unique_value=self.train_df[cat_col].unique()
                for value in unique_value:
                    tmp_df=self.train_df[self.train_df[cat_col]==value]
                    print(f"{cat_col}=={value}:mean_{num_col}=={tmp_df[num_col].mean()}")
                print("-"*50)
            print()
        print("num_cols VS num_cols:")
        for i in range(len(self.num_cols)):
            for j in range(i+1,len(self.num_cols)):
                x1=self.train_df[self.num_cols[i]].values
                x2=self.train_df[self.num_cols[j]].values
                if abs(self.pearson_corr(x1,x2))>0.9:
                    print(f"{self.num_cols[i]} and {self.num_cols[j]} have strong linear correlation.")
                    
        #如果是分类任务
        if self.train_df[self.TARGET_NAME].nunique()<50:
            print("TARGET VS cat_cols:")
            unique_target=self.train_df[self.TARGET_NAME].unique()
            for target in unique_target:
                for cat_col in self.cat_cols:
                    unique_value=self.train_df[cat_col].unique()
                    for value in unique_value:
                        #当cat_col==value时,target有没有唯一值
                        tmp_df=self.train_df[self.train_df[cat_col]==value]
                        #如果target基本只有它一个值
                        if len(tmp_df[tmp_df[self.TARGET_NAME]==target])>0.99*len(tmp_df):
                            print(f"when {cat_col}={value},{self.TARGET_NAME}={target}")
                            
                        tmp_df=self.train_df[self.train_df[self.TARGET_NAME]==target]
                        if len(tmp_df[tmp_df[cat_col]==value])>0.99*len(tmp_df):
                            print(f"when {self.TARGET_NAME}={target},{cat_col}={value}")   
            print("-"*50)
            print("TARGET VS num_cols:")
            for num_col in self.num_cols:
                for target in unique_target:
                    tmp_df=self.train_df[self.train_df[self.TARGET_NAME]==target]
                    print(f"when {self.TARGET_NAME}={target},mean_{num_col}={tmp_df[num_col].mean()}")  
                print("-"*50)      
        else:#如果是回归任务的话,那肯定是数值类型的target
            print("TARGET VS cat_cols:")
            for target in unique_target:
                for cat_col in self.cat_cols:
                    unique_value=self.train_df[cat_col].unique()
                    for value in unique_value:
                        tmp_df=self.train_df[self.train_df[cat_col]==value]
                        print(f"when {cat_col}={value},mean_{self.TARGET_NAME}={tmp_df[self.TARGET_NAME].mean()}")  
                print("-"*50)   
            
                
    
    #对训练和测试数据某列字符串列构造tf-idf特征,取n个特征,最后降维为p个特征
    def tf_idf(self,train, test, column,n,p):
        print(f"tf-idf done with {column}")
        vectorizer=TfidfVectorizer(max_features=n)#创建tf-idf模型,取出最重要的n个特征
        #对训练数据的列进行fit,得到训练数据和测试数据的向量.
        vectors_train=vectorizer.fit_transform(train[column])
        vectors_test=vectorizer.transform(test[column])

        svd=TruncatedSVD(p)#创建截断SVD,降维到P维
        x_pca_train=svd.fit_transform(vectors_train)#对训练向量和测试向量进行降维
        x_pca_test=svd.transform(vectors_test)
        tfidf_df_train=pd.DataFrame(x_pca_train)#转成表格型数据
        print(len(tfidf_df_train))
        tfidf_df_test=pd.DataFrame(x_pca_test)
        
        #对列名进行调整 ‘Surname_tfidf_{idx}’
        cols=[(column+"_tfidf_"+str(f)) for f in tfidf_df_train.columns]
        tfidf_df_train.columns=cols
        tfidf_df_test.columns=cols
        #按列拼接在一起.
        train=pd.concat([train,tfidf_df_train], axis=1)
        test=pd.concat([test,tfidf_df_test], axis=1)
        train.drop([column],axis=1,inplace=True)
        test.drop([column],axis=1,inplace=True)
        return train, test
    #对类别型字符串进行独热编码
    def one_hot_encoder(self,total_df):
        print(f"one hot encoder with {self.cat_cols}")
        for col in self.cat_cols:
            if total_df[col].nunique()==2:#如果类别数量等于2的话,有一列onehot就行了.
                value=total_df[col].values[0]
                total_df[col+"_"+str(value)]=(total_df[col]==value)
                total_df.drop([col],axis=1,inplace=True)
            else:#如果类别数量很多的话,一个一个来
                values=total_df[col].unique()
                for value in values:
                    total_df[col+"_"+str(value)]=(total_df[col]==value)
                total_df.drop([col],axis=1,inplace=True)
        return total_df          
    #这个可能每次比赛都要具体情况具体分析.
    def make_feats(self):
        total_df=pd.concat((self.train_df,self.test_df),axis=0)
        print(f"len(total_df):{len(total_df)}")
        total_df=self.one_hot_encoder(total_df)
        
#         #增加特征BMI,将部分特征保留合适的小数位数
#         total_df['Age']=round(total_df['Age'],2)
#         total_df['Height']=round(total_df['Height'],2)
#         total_df['Weight']=round(total_df['Weight'],2)
#         total_df['BMI']=total_df['Weight']/(total_df['Height']**2)
#         cols_to_round = ['FCVC',"NCP","CH2O","FAF","TUE"]
#         for col in cols_to_round:
#             total_df[col] = round(total_df[col])
#             total_df[col] = total_df[col].astype('int')        
        
        
        print("-"*50)
        train_feats=total_df[:len(self.train_df)].reset_index(drop=True)
        test_feats=total_df[len(self.train_df):].reset_index(drop=True)
        print(len(train_feats))
        for col in self.str_cols:
            train_feats,test_feats=self.tf_idf(train_feats, test_feats, column=col,n=int(0.5*train_feats[col].nunique()),p=5)
            print(f"len(train_feats):{len(train_feats)}")
        print("-"*50)
        print(f"Done!! total_features:{len(train_feats.keys().values)-1}")
        return train_feats,test_feats

    
    def submission(self,submission_path=None,test_pred=None):
        if submission_path!=None:#一般都有训练数据的路径
            self.submission_df=pd.read_csv(submission_path)
        else:#没有提交文件的路径就报错
            assert 0#assert 0==1?
        self.submission_df[self.TARGET_NAME]=test_pred
        self.submission_df.to_csv("submission.csv",index=None)
        return self.submission_df

### Feature engineer

In [ ]:
print("Import data.")
preprocessor=Preprocessor(Config.train_path,Config.origin_path,Config.test_path,Config.TARGET_NAME,Config.seed)
print("-"*50)
print("EDA!!!")
preprocessor.EDA()
print("-"*50)
print("make features.")
train_feats,test_feats=preprocessor.make_feats()
print("-"*50)
print("labels to idx")
labels=['Insufficient_Weight','Normal_Weight', 'Overweight_Level_I','Overweight_Level_II','Obesity_Type_I','Obesity_Type_II', 'Obesity_Type_III'] 
labels2idx={}
idx2labels={}
for idx in range(len(labels)): 
    labels2idx[labels[idx]]=idx 
    idx2labels[idx]=labels[idx]
train_feats['NObeyesdad']=train_feats['NObeyesdad'].apply(lambda x: labels2idx[x])

## Model training

In [ ]:
#model
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier

#KFold是直接分成k折,StratifiedKFold还要考虑每种类别的占比
from sklearn.model_selection import StratifiedKFold

def fit_and_predict(model):
    X=train_feats.drop([Config.TARGET_NAME],axis=1).copy()
    y=train_feats[Config.TARGET_NAME].copy()
    test_X=test_feats.drop([Config.TARGET_NAME],axis=1).copy()
    oof_pred_pro=np.zeros((len(X),np.max(y)+1))
    test_pred_pro=np.zeros((Config.num_folds,len(test_X),np.max(y)+1))
    #10折交叉验证
    skf = StratifiedKFold(n_splits=Config.num_folds,random_state=Config.seed, shuffle=True)

    for fold, (train_index, valid_index) in (enumerate(skf.split(X, y.astype(str)))):
        print(f"fold:{fold}")

        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
        
        model.fit(X_train,y_train)
        
        oof_pred_pro[valid_index]=model.predict_proba(X_valid)
        test_pred_pro[fold]=model.predict_proba(test_X)
    
    return oof_pred_pro,test_pred_pro

#选择的参数都来自这个notebook https://www.kaggle.com/code/ksevta/ps4e2-xgb-lgbm-0-92/notebook
lgb_params={
    "objective": "multiclass",          # Objective function for the model
    "metric": "multi_logloss",          # Evaluation metric
    "verbosity": -1,                    # Verbosity level (-1 for silent)
    "boosting_type": "gbdt",            # Gradient boosting type
    "random_state": Config.seed,       # Random state for reproducibility
    "num_class": 7,                     # Number of classes in the dataset
    'learning_rate': 0.030962211546832760,  # Learning rate for gradient boosting
    'n_estimators': 500,                # Number of boosting iterations
    'lambda_l1': 0.009667446568254372,  # L1 regularization term
    'lambda_l2': 0.04018641437301800,   # L2 regularization term
    'max_depth': 10,                    # Maximum depth of the trees
    'colsample_bytree': 0.40977129346872643,  # Fraction of features to consider for each tree
    'subsample': 0.9535797422450176,    # Fraction of samples to consider for each boosting iteration
    'min_child_samples': 26             # Minimum number of data needed in a leaf
}

xgb_params = {'grow_policy': 'depthwise', 'n_estimators': 982, 
               'learning_rate': 0.050053726931263504, 'gamma': 0.5354391952653927, 
               'subsample': 0.7060590452456204, 'colsample_bytree': 0.37939433412123275, 
               'max_depth': 23, 'min_child_weight': 21, 'reg_lambda': 9.150224029846654e-08,
               'reg_alpha': 5.671063656994295e-08,
               'booster':'gbtree','objective':'multi:softmax',"verbosity":0
              }
cat_params={'learning_rate': 0.13762007048684638, 'depth': 5, 
          'l2_leaf_reg': 5.285199432056192, 'bagging_temperature': 0.6029582154263095,
         'random_seed': Config.seed,'verbose': False,'iterations':1000}

print("random forest model")
rf_oof_pred_pro,rf_test_pred_pro=fit_and_predict(model=RandomForestClassifier(random_state=Config.seed))
print("lgb model")
lgb_oof_pred_pro,lgb_test_pred_pro=fit_and_predict(model= LGBMClassifier(**lgb_params,verbose=-1))
print("xgb model")
xgb_oof_pred_pro,xgb_test_pred_pro=fit_and_predict(model= XGBClassifier(**xgb_params,seed=Config.seed))
print("cat model")
cat_oof_pred_pro,cat_test_pred_pro=fit_and_predict(model= CatBoostClassifier(**cat_params))

## Blending

In [ ]:
#评估指标是log_loss
def log_loss(y_true,y_pred):
    eps=10**(-15)
    y_true=np.clip(y_true,eps,1-eps)
    y_pred=np.clip(y_pred,eps,1-eps)
    return -np.mean(np.sum(y_true*np.log(y_pred),axis=-1))
def accuracy(y_true,y_pred):
    return np.mean(y_true==y_pred)
oof_target=train_feats[Config.TARGET_NAME].values
oof_one_hot=np.eye(np.max(oof_target)+1)[oof_target]
step=100
best_w1=1
best_w2=1
best_w3=1
best_accuracy=0
best_log_loss=10
for w1 in range(1,step-1,1):
    for w2 in range(1,step-w1-1,1):
        for w3 in range(1,step-w1-w2-1,1):
            blend_oof_pred_pro=(w1*rf_oof_pred_pro+w2*lgb_oof_pred_pro+w3*xgb_oof_pred_pro+(step-w1-w2-w3)*cat_oof_pred_pro)/step
            current_log_loss=log_loss(oof_one_hot,blend_oof_pred_pro)
            current_accuracy=accuracy(oof_target,np.argmax(blend_oof_pred_pro,axis=1))
            if current_accuracy>best_accuracy:
                best_w1=w1
                best_w2=w2
                best_w3=w3
                best_accuracy=current_accuracy
                best_log_loss=current_log_loss
                print(f"best_w1:{best_w1},best_w2:{best_w2},best_w3:{best_w3},best_accuracy:{best_accuracy},best_log_loss:{best_log_loss}")
            elif (current_accuracy==best_accuracy) and current_log_loss<best_log_loss:
                best_w1=w1
                best_w2=w2
                best_w3=w3
                best_log_loss=current_log_loss
                print(f"best_w1:{best_w1},best_w2:{best_w2},best_w3:{best_w3},best_accuracy:{best_accuracy},best_log_loss:{best_log_loss}")

## Submission

In [ ]:
blend_test_pred_pro=(best_w1*rf_test_pred_pro+best_w2*lgb_test_pred_pro+best_w3*xgb_test_pred_pro+(step-best_w1-best_w2-best_w3)*cat_test_pred_pro)/step
test_pred=np.argmax(blend_test_pred_pro.mean(axis=0),axis=1)
test_pred=[idx2labels[int(pred)] for pred in test_pred]
print("submission")
submission_df=preprocessor.submission(submission_path=Config.submission_path,test_pred=test_pred)
submission_df.head()